### Test a trained Agent

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Dense, concatenate, Flatten, Conv2D, Permute, Reshape
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import History 
from keras.utils import np_utils, multi_gpu_model

import sys
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from IPython import get_ipython

from fittingEnv import *
from interface import *

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

def get_new_audio ():
        path = 'PATH_TO_NOISY_SPEECH_FILES'
        filename = glob.glob(os.path.join(path, '*.wav'))
        seed = random.randint(0, (1<< 32) - 1)
        random.seed(seed)
        rnd_idx  = randint(0, np.asarray(filename).shape[0]-1)
        rnd_idx = 100
        speech_file  = filename[rnd_idx]
        audio, fs = librosa.load(speech_file)
        fsTarget = int(16000.0)
        if len(audio.shape) > 1:
            audio = audio[:,0]
        if fs != fsTarget:
            audio = librosa.resample(audio, fs, fsTarget)
        
        return audio, fsTarget
    
def next_observation(audio, fs=16000, overlap_size=int(0.01*16000), frame_size=int(0.02*16000), nFilt=80, nFrames  = 3 ):


    ## Compute mel spectrogram features from uncompressed audio
    audio_len  = int(fs * 2.5) # only 2.5 seconds audio for agent input
    audio_seg  = audio[:audio_len]
    MFSC_feat_audio = librosa.feature.melspectrogram(y= audio_seg, sr=fs, hop_length=overlap_size, 
                                                     win_length=frame_size, window='hann', n_mels=nFilt, fmax=8000)
    mellog = librosa.power_to_db(MFSC_feat_audio, ref=np.max)
    # Normalization each row to [0,1]:
    for i in range(nFilt):
        arr = mellog[i,:]
        arr = arr - np.min(arr)
        safe_max = np.max(np.abs(arr))
        if safe_max==0:
            safe_max = 1
        arr = arr / safe_max
        mellog[i,:] = arr

    ## Create Mel spectrogram images
    # nFrames  = int(np.floor(MFSC_features.shape[1]/self.nFilt))
    mfsc_img_audio = np.zeros((nFilt, nFilt, nFrames))
    k = 0
    for i in range(nFrames):
        mfsc_img_audio[:,:,i] = mellog[:,k:k+nFilt]
        k = k + nFilt

    # Update system states
    obs_agent = mfsc_img_audio ## TODO2 : include newCR or action as well in the next obs.

    return obs_agent, mellog


def apply_compression (audio , newCR, INIT_softG, fs=16000):

    def npArray2Matlab (arr):
        return matlab.double(arr.tolist()) # casting a as list

    audio_len  = int(fs * 2.5) # only 2.5 seconds audio for agent input
    audio_seg  = audio[:audio_len]
    out_DRC = eng.perform_compression (npArray2Matlab(audio_seg), npArray2Matlab(newCR), fs , npArray2Matlab(INIT_softG), nargout=1 )
    out_DRC = np.array(out_DRC)

    # normalize the compressued audio (amplitude compensation)
    audio_max1 = np.max(np.abs(out_DRC))
    if audio_max1 == 0:
        audio_max1 = 1
    out_DRC = out_DRC / audio_max1
    out_DRC = out_DRC.reshape(-1)

    return out_DRC


### Load trained RL agent model

In [ ]:
model = load_model("rl_model.h5")
print("Loaded model from disk")     
model._make_predict_function() 

### Collect hearing perefrences (Visit 3)

In [ ]:
# window allocation for human interface :
if QApplication.instance() is None:
    app = QApplication(sys.argv)
%gui qt5 ## enable PyQt5 event loop integration for .pynb files
#get_ipython().run_line_magic('gui', 'qt5 ## enable PyQt5 event loop integration') 

In [ ]:

INITIAL_CRs   = np.array ([1.1, 1.2, 1.3, 1.2, 1.4]) # Define 2D matrix numpy array
INITIAL_softG = np.array ([6.0, 11.0, 20.0, 23.0, 20.0]) # Define 2D matrix numpy array

env = fittingEnv()

preference = -1 # initialize to none
adjustCR = [1,1,1,1,1] # only for the first time

for i in range(30):

    while True:

        audio,fs = get_new_audio ()
        obs_agent, _ = next_observation(audio, fs)

        adjustCR = np.reshape(adjustCR, (5,))

        # Agent obs is # mel-spectrogram images of unprocessed audio + prev actions
        obs1 = np.expand_dims(obs_agent, axis=0)
        obs2 = np.expand_dims(adjustCR, axis=0)
        #print(obs1.shape)
        #print(obs2.shape)

        pred_agent = model.predict([obs1 , obs2]) # gets predicted action
        action = np.argmax(pred_agent)

        obs_agent, obs_RePrd, audio, adjustCR, _, done, info = env.step(action)
        print(adjustCR)

        newCR   = np.multiply(np.reshape(INITIAL_CRs, (5,)), np.reshape(adjustCR, (5,)) )
        print("newCR = ", newCR) 

        out_DRC_rl  = apply_compression (audio , newCR, INITIAL_softG, fs)
        audio_max1 = np.max(np.abs(out_DRC_rl))
        if audio_max1 == 0:
            audio_max1 = 1
        out_DRC_rl = out_DRC_rl / audio_max1
        
        out_DRC_DSL = apply_compression (audio , INITIAL_CRs, INITIAL_softG, fs)
        audio_max2 = np.max(np.abs(out_DRC_DSL))
        if audio_max2 == 0:
            audio_max2 = 1
        out_DRC_DSL = out_DRC_DSL / audio_max2

        idx = randint(1,2)
        print(idx)
        if idx == 1:
            audio1 = out_DRC_DSL
            audio2 = out_DRC_rl
        else:
            audio1 = out_DRC_rl
            audio2 = out_DRC_DSL
        #########
        data, fs2 = sf.read('beep-01a.wav', dtype='float32')  
        sd.play(0.1*data, fs2)
        status = sd.wait()  # Wait until file is done playing
        #########
        print("Please choose the preferred audio, 1 or 2. Press both for neutral.")
        win = AudioPlayer()
        win.show()
        win.set_audio(audio1, fs, 0)
        win.set_audio(audio2, fs, 1)
        key = win.get_response()
        win.close()
        print("preference is = ", key)

        if key == 0:
            preference = 0
        elif key == 1:
            preference = 1
        elif key == 2:
            preference = 2

        # Open a file to have users preferences
        f = open("test_results.txt", "a")
        if preference == 0:
            if idx ==1:
                pref_save = [1.0,0.0] 
            else:
                pref_save = [0.0,1.0] 
        elif preference == 1:
            if idx ==1:
                pref_save = [0.0,1.0] 
            else:
                pref_save = [1.0,0.0]
        else:
            pref_save = [0.5,0.5]   

        print('CR1 = ', np.reshape(INITIAL_CRs, (5,)), file=f) # save the preference into .txt file
        print('CR2 = ', newCR, file=f) # save the preference into .txt file
        print('preference = ', pref_save, file=f) # save the preference into .txt file
        print('====================================', file=f)
        f.close()

        # End of saving preferences
        if preference != -1:
            break


print(" ******** End of session! :)  ********")